In [1]:
from Bio import pairwise2
from Bio.SubsMat import MatrixInfo
import sys

/home/damiano/miniconda3/lib/python3.9/site-packages/Bio/SubsMat/__init__.py:126: BiopythonDeprecationWarning: Bio.SubsMat has been deprecated, and we intend to remove it in a future release of Biopython. As an alternative, please consider using Bio.Align.substitution_matrices as a replacement, and contact the Biopython developers if you still need the Bio.SubsMat module.
  warnings.warn(


In [2]:
# Fasta
disprot_new_fasta_file = "../data/output/homology/disprot_new.fasta"
disprot_old_fasta_file = "../data/output/homology/disprot_old.fasta"
seqres_fasta_file = "../data/output/homology/pdb_seqres.txt"

# Blast
disprot_new_old_blast_file = "../data/output/homology/disprot_new_old.blast"
disprot_new_pdb_blast_file = "../data/output/homology/disprot_new_pdb.blast"

# Output
out_file = "../data/output/homology/homology.tsv"

In [3]:
# Collect all sequences
sequences = {}
for file_name in [disprot_new_fasta_file, disprot_old_fasta_file, seqres_fasta_file]:
    with open(file_name) as f:
        for line in f:
            if line[0] == ">":
                name = line[1:].strip().split()[0]
            else:
                sequences[name] = line.strip()

print(list(sequences.keys())[:10])
print(len(sequences))

['DP02342|P06837', 'DP02348|Q8N5F7', 'DP02361|O95429', 'DP02364|Q9H6Z4-3', 'DP02376|Q4ACU6', 'DP02377|Q9GRZ3', 'DP02393|Q8IZD2', 'DP02401|O61380', 'DP02405|Q8IYW5', 'DP02411|Q08050']
755574


In [4]:
# Parse blast output
# Recalculate identity percentage normalizing over the query length

"""
DP02150 DP02150 100.000 462     0       0       1       462     1       462     0.0     939
DP02150 DP01437 37.895  95      58      1       288     382     199     292     1.58e-12        64.7
DP02150 DP02426 43.878  98      49      4       262     356     115     209     1.16e-10        60.5
DP02849	DP00563	45.833	24	9	1	144	163	139	162	1.7	25.8
"""

blast = {}

for blast_file, db in zip([disprot_new_old_blast_file, disprot_new_pdb_blast_file], 
                                ["disprot", "pdb"]):
    with open(blast_file) as f:
        for line in f:
            qseqid, sseqid, pident, length, mismatch, gapopen, qstart, qend, sstart, send, evalue, bitscore = line.strip().split()
            pident = (float(qend) - float(qstart) + 1 - float(mismatch))*100 / len(sequences[qseqid])

            blast.setdefault(qseqid, {}).setdefault(db, {}).setdefault(sseqid, 0.0)
            blast[qseqid][db][sseqid] = max(blast[qseqid][db][sseqid], float(pident))
    print(blast_file, len(blast))

../data/output/homology/disprot_new_old.blast 348
../data/output/homology/disprot_new_pdb.blast 348


In [5]:
with open(out_file, "w") as fout:

    fout.write("disprot_id\tdb\tblast_acc\tblast_id\tlocal_acc\tlocal_id\tglobal_acc\tglobal_id\n")

    for i, disprot_id in enumerate(blast):
        print(i, disprot_id)
        
        disprot_id_len = len(sequences[disprot_id])
        
        for db in blast[disprot_id]:
            
            matches = {"local": [], "global": []}
            
            for sseqid in blast[disprot_id][db]:
#                 print(sequences[sseqid])
                try:
                    alignment = pairwise2.align.globalds(sequences[disprot_id], sequences[sseqid], MatrixInfo.blosum62, -10, -0.5, one_alignment_only=True)[0]
                    identity = sum([1 if a == b else 0 for a, b in zip(alignment.seqA, alignment.seqB)]) * 100 / disprot_id_len
                    matches["global"].append((sseqid, identity))
                except Exception as e:
                    # print(e)
                    pass
                
                try:
                    alignment = pairwise2.align.localds(sequences[disprot_id], sequences[sseqid], MatrixInfo.blosum62, -10, -0.5, one_alignment_only=True)[0]
                    identity = sum([1 if a == b else 0 for a, b in zip(alignment.seqA, alignment.seqB)]) * 100 / disprot_id_len
                    matches["local"].append((sseqid, identity))
                except Exception as e:
                    # print(e)
                    pass

            # Sort based on best identity
            best_blast = sorted(blast[disprot_id][db].items(), key=lambda x: x[1], reverse=True)[0]
            best_local = [None, None]
            best_global = [None, None]
            if matches["local"]:
                best_local = sorted(matches["local"], key=lambda x: x[1], reverse=True)[0]
            if matches["global"]:
                best_global = sorted(matches["global"], key=lambda x: x[1], reverse=True)[0]
            
#             print(best_blast, best_local, best_global)
            
            # Write to file
            fout.write("{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\n".format(disprot_id, db, *best_blast, *best_local, *best_global))
            
#         if i==2:
#             break


0 DP02342|P06837
1 DP02348|Q8N5F7
2 DP02361|O95429
3 DP02364|Q9H6Z4-3
4 DP02376|Q4ACU6
5 DP02377|Q9GRZ3
6 DP02393|Q8IZD2
7 DP02401|O61380
8 DP02405|Q8IYW5
9 DP02411|Q08050
10 DP02418|P35568
11 DP02421|O94687
12 DP02445|Q96RS0
13 DP02448|Q03707
14 DP02449|Q15583
15 DP02457|O95400
16 DP02465|P07199
17 DP02468|P17480
18 DP02470|Q12495
19 DP02472|Q3U182
20 DP02532|Q07866
21 DP02535|Q8R4T5
22 DP02537|P30130
23 DP02542|P39061
24 DP02543|Q9NUM4
25 DP02544|Q04410
26 DP02546|P16070
27 DP02553|Q9C636
28 DP02556|P09077
29 DP02557|P07548
30 DP02558|Q9NP70
31 DP02559|Q969R2
32 DP02567|P01266
33 DP02571|P80192
34 DP02585|Q9H900
35 DP02595|P25490
36 DP02601|Q96NW4
37 DP02604|P20020-1
38 DP02606|P13647
39 DP02612|Q9Y6D6
40 DP02627|Q8TEP8
41 DP02628|O94986
42 DP02649|P01023
43 DP02658|Q8N807
44 DP02671|P38111
45 DP02674|P54368
46 DP02681|B8ZXI1
47 DP02686|Q6XHB2
48 DP02688|O96759
49 DP02691|Q92608
50 DP02695|Q9JIP3
51 DP02699|Q6V1X1
52 DP02718|Q8ST83
53 DP02728|P42166
54 DP02731|Q5U651
55 DP02732|Q8IWJ